📝 **Author:** Amirhossein Heydari - 📧 **Email:** <amirhosseinheydari78@gmail.com> - 📍 **Origin:** [mr-pylin/media-processing-workshop](https://github.com/mr-pylin/media-processing-workshop)

---


**Table of contents**<a id='toc0_'></a>    
- [Dependencies](#toc1_)    
- [Load Images](#toc2_)    
- [Image Interpolation](#toc3_)    
  - [Nearest Neighbor Interpolation](#toc3_1_)    
    - [Manual](#toc3_1_1_)    
    - [Using OpenCV](#toc3_1_2_)    
    - [Using PIL](#toc3_1_3_)    
  - [Bilinear Interpolation](#toc3_2_)    
    - [Manual](#toc3_2_1_)    
    - [Using OpenCV](#toc3_2_2_)    
    - [Using PIL](#toc3_2_3_)    
  - [Fourier Transform Interpolation](#toc3_3_)    
    - [Manual](#toc3_3_1_)    
  - [Comparison](#toc3_4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Dependencies](#toc0_)


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.gridspec import GridSpec
from PIL import Image

# <a id='toc2_'></a>[Load Images](#toc0_)


In [ ]:
im_1 = cv2.imread("../assets/images/dip_3rd/CH02_Fig0222(b)(cameraman).tif", flags=cv2.IMREAD_GRAYSCALE)
im_2 = cv2.cvtColor(
    cv2.imread("../assets/images/dip_3rd/CH06_Fig0638(a)(lenna_RGB).tif"),
    cv2.COLOR_BGR2RGB,
)

In [ ]:
img_1 = Image.fromarray(im_1)
img_2 = Image.fromarray(im_2)

In [ ]:
# plot
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(8, 4), layout="compressed")
axs[0].imshow(im_1, cmap="gray")
axs[0].set_title("CH02_Fig0222(b)(cameraman).tif")
axs[0].axis("off")
axs[1].imshow(im_2)
axs[1].set_title("CH06_Fig0638(a)(lenna_RGB).tif")
axs[1].axis("off")
plt.show()

# <a id='toc3_'></a>[Image Interpolation](#toc0_)

It refers to the “guess” of intensity values at missing locations When resizing an image [Mostly in Down Scaling]

| Interpolation Method | Downscaling Quality | Upscaling Quality | Performance   |
| -------------------- | ------------------- | ----------------- | ------------- |
| Nearest Neighbor     |                     |                   | ⭐⭐⭐⭐⭐ |
| Box                  | ⭐                 |                   | ⭐⭐⭐⭐    |
| Bilinear             | ⭐                 | ⭐                | ⭐⭐⭐      |
| Hamming              | ⭐⭐               |                   | ⭐⭐⭐      |
| Bicubic              | ⭐⭐⭐            | ⭐⭐⭐           | ⭐⭐        |
| Lanczos              | ⭐⭐⭐⭐          | ⭐⭐⭐⭐        | ⭐           |
| Fourier Transform    | ⭐⭐⭐⭐          | ⭐⭐⭐⭐        | ⭐           |

Reference [except Fouriet Transform]: [pillow.readthedocs.io/en/stable/handbook/concepts.html#filters-comparison-table](https://pillow.readthedocs.io/en/stable/handbook/concepts.html#filters-comparison-table)

📝 **Docs**:

- `PIL.Image.Image.resize`: [pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.resize](https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.resize)
- Filters: [pillow.readthedocs.io/en/stable/handbook/concepts.html#concept-filters](https://pillow.readthedocs.io/en/stable/handbook/concepts.html#concept-filters)
- `cv2.resize`: [docs.opencv.org/5.x/da/d54/group__imgproc__transform.html#ga47a974309e9102f5f08231edc7e7529d](https://docs.opencv.org/5.x/da/d54/group__imgproc__transform.html#ga47a974309e9102f5f08231edc7e7529d)
- `InterpolationFlags`: [docs.opencv.org/5.x/da/d54/group__imgproc__transform.html#gga5bb5a1fea74ea38e1a5445ca803ff121ac97d8e4880d8b5d509e96825c7522deb](https://docs.opencv.org/5.x/da/d54/group__imgproc__transform.html#gga5bb5a1fea74ea38e1a5445ca803ff121ac97d8e4880d8b5d509e96825c7522deb)


## <a id='toc3_1_'></a>[Nearest Neighbor Interpolation](#toc0_)


### <a id='toc3_1_1_'></a>[Manual](#toc0_)


In [ ]:
def nearest_neighbor_interpolation(image: np.ndarray, new_height: int, new_width: int) -> np.ndarray:
    height, width = image.shape[:2]
    new_image = np.zeros((new_height, new_width, *image.shape[2:]))
    x_ratio = width / new_width
    y_ratio = height / new_height

    for i in range(new_height):
        for j in range(new_width):
            x = int(j * x_ratio)
            y = int(i * y_ratio)
            new_image[i, j] = image[y, x]

    return new_image.astype(image.dtype)

In [ ]:
im1_nni_1 = nearest_neighbor_interpolation(im_1, new_height=128, new_width=128)
im1_nni_2 = nearest_neighbor_interpolation(im_1, new_height=32, new_width=32)
im1_nni_3 = nearest_neighbor_interpolation(im_1, new_height=555, new_width=555)
im1_nni_4 = nearest_neighbor_interpolation(im_1, new_height=256, new_width=128)
im1_nni_5 = nearest_neighbor_interpolation(im_1, new_height=128, new_width=256)
im1_nni_6 = nearest_neighbor_interpolation(im_1, new_height=64, new_width=512)

In [ ]:
# plot
fig = plt.figure(figsize=(16, 10), layout="compressed")
gs = GridSpec(nrows=2, ncols=4, figure=fig)
axes = [fig.add_subplot(i) for i in [gs[:, 0], gs[0, 1], gs[0, 2], gs[0, 3], gs[1, 1], gs[1, 2], gs[1, 3]]]
images = [im_1, im1_nni_1, im1_nni_2, im1_nni_3, im1_nni_4, im1_nni_5, im1_nni_6]
titles = [
    f"Original {im_1.shape}",
    f"{im1_nni_1.shape} [down scaled]",
    f"{im1_nni_2.shape} [down scaled]",
    f"{im1_nni_3.shape} [up Scaled]",
    f"{im1_nni_4.shape}",
    f"{im1_nni_5.shape}",
    f"{im1_nni_6.shape}",
]
for ax, img, title in zip(axes, images, titles):
    ax.imshow(img, cmap="gray", vmin=0, vmax=255)
    ax.set_title(title)
plt.show()

### <a id='toc3_1_2_'></a>[Using OpenCV](#toc0_)


In [ ]:
im2_nni_1 = cv2.resize(im_2, dsize=(128, 128), interpolation=cv2.INTER_NEAREST_EXACT)
im2_nni_2 = cv2.resize(im_2, dsize=(32, 32), interpolation=cv2.INTER_NEAREST_EXACT)
im2_nni_3 = cv2.resize(im_2, dsize=(555, 555), interpolation=cv2.INTER_NEAREST_EXACT)
im2_nni_4 = cv2.resize(im_2, dsize=(256, 128), interpolation=cv2.INTER_NEAREST_EXACT)
im2_nni_5 = cv2.resize(im_2, dsize=(128, 256), interpolation=cv2.INTER_NEAREST_EXACT)
im2_nni_6 = cv2.resize(im_2, dsize=(64, 512), interpolation=cv2.INTER_NEAREST_EXACT)

In [ ]:
# plot
fig = plt.figure(figsize=(16, 8), layout="compressed")
gs = GridSpec(nrows=2, ncols=4, figure=fig)
axes = [fig.add_subplot(i) for i in [gs[:, 0], gs[0, 1], gs[0, 2], gs[0, 3], gs[1, 1], gs[1, 2], gs[1, 3]]]
images = [im_2, im2_nni_1, im2_nni_2, im2_nni_3, im2_nni_4, im2_nni_5, im2_nni_6]
titles = [
    f"Original {im_2.shape[:2]}",
    f"{im2_nni_1.shape[:2]} [down scaled]",
    f"{im2_nni_2.shape[:2]} [down scaled]",
    f"{im2_nni_3.shape[:2]} [up Scaled]",
    f"{im2_nni_4.shape[:2]}",
    f"{im2_nni_5.shape[:2]}",
    f"{im2_nni_6.shape[:2]}",
]
for ax, img, title in zip(axes, images, titles):
    ax.imshow(img, cmap="gray", vmin=0, vmax=255)
    ax.set_title(title)
plt.show()

### <a id='toc3_1_3_'></a>[Using PIL](#toc0_)


In [ ]:
im2_nni_1 = img_2.resize(size=(128, 128), resample=Image.Resampling.NEAREST)
im2_nni_2 = img_2.resize(size=(32, 32), resample=Image.Resampling.NEAREST)
im2_nni_3 = img_2.resize(size=(555, 555), resample=Image.Resampling.NEAREST)
im2_nni_4 = img_2.resize(size=(256, 128), resample=Image.Resampling.NEAREST)
im2_nni_5 = img_2.resize(size=(128, 256), resample=Image.Resampling.NEAREST)
im2_nni_6 = img_2.resize(size=(64, 512), resample=Image.Resampling.NEAREST)

In [ ]:
# plot
fig = plt.figure(figsize=(16, 8), layout="compressed")
gs = GridSpec(2, 4, figure=fig)
axes = [fig.add_subplot(i) for i in [gs[:, 0], gs[0, 1], gs[0, 2], gs[0, 3], gs[1, 1], gs[1, 2], gs[1, 3]]]
images = [im_2, im2_nni_1, im2_nni_2, im2_nni_3, im2_nni_4, im2_nni_5, im2_nni_6]
titles = [
    f"Original {im_2.size}",
    f"{im2_nni_1.size} [down scaled]",
    f"{im2_nni_2.size} [down scaled]",
    f"{im2_nni_3.size} [up Scaled]",
    im2_nni_4.size,
    im2_nni_5.size,
    im2_nni_6.size,
]
for ax, img, title in zip(axes, images, titles):
    ax.imshow(img, cmap="gray", vmin=0, vmax=255)
    ax.set_title(title)
plt.show()

## <a id='toc3_2_'></a>[Bilinear Interpolation](#toc0_)


### <a id='toc3_2_1_'></a>[Manual](#toc0_)


In [ ]:
def bilinear_interpolation(image: np.ndarray, new_height: int, new_width: int) -> np.ndarray:
    height, width = image.shape[:2]
    new_image = np.zeros((new_height, new_width, *image.shape[2:]))
    x_ratio = float(width - 1) / new_width
    y_ratio = float(height - 1) / new_height

    for i in range(new_height):
        for j in range(new_width):
            x = j * x_ratio
            y = i * y_ratio
            x1, y1 = int(x), int(y)
            x2, y2 = min(x1 + 1, width - 1), min(y1 + 1, height - 1)

            dx = x - x1
            dy = y - y1

            new_image[i, j] = (
                (1 - dx) * (1 - dy) * image[y1, x1] + dx * (1 - dy) * image[y1, x2] + (1 - dx) * dy * image[y2, x1] + dx * dy * image[y2, x2]
            )

    return new_image.astype(image.dtype)

In [ ]:
im1_bli_1 = bilinear_interpolation(im_1, new_height=128, new_width=128)
im1_bli_2 = bilinear_interpolation(im_1, new_height=32, new_width=32)
im1_bli_3 = bilinear_interpolation(im_1, new_height=555, new_width=555)
im1_bli_4 = bilinear_interpolation(im_1, new_height=256, new_width=128)
im1_bli_5 = bilinear_interpolation(im_1, new_height=128, new_width=256)
im1_bli_6 = bilinear_interpolation(im_1, new_height=64, new_width=512)

In [ ]:
# plot
fig = plt.figure(figsize=(16, 8), layout="compressed")
gs = GridSpec(2, 4, figure=fig)
axes = [fig.add_subplot(i) for i in [gs[:, 0], gs[0, 1], gs[0, 2], gs[0, 3], gs[1, 1], gs[1, 2], gs[1, 3]]]
images = [im_1, im1_bli_1, im1_bli_2, im1_bli_3, im1_bli_4, im1_bli_5, im1_bli_6]
titles = [
    f"Original {im_2.shape[:2]}",
    f"{im1_bli_1.shape} [down scaled]",
    f"{im1_bli_2.shape} [down scaled]",
    f"{im1_bli_3.shape} [up Scaled]",
    im1_bli_4.shape,
    im1_bli_5.shape,
    im1_bli_6.shape,
]
for ax, img, title in zip(axes, images, titles):
    ax.imshow(img, cmap="gray", vmin=0, vmax=255)
    ax.set_title(title)
plt.show()

### <a id='toc3_2_2_'></a>[Using OpenCV](#toc0_)


In [ ]:
im2_bli_1 = cv2.resize(im_2, dsize=(128, 128), interpolation=cv2.INTER_LINEAR)
im2_bli_2 = cv2.resize(im_2, dsize=(32, 32), interpolation=cv2.INTER_LINEAR)
im2_bli_3 = cv2.resize(im_2, dsize=(555, 555), interpolation=cv2.INTER_LINEAR)
im2_bli_4 = cv2.resize(im_2, dsize=(256, 128), interpolation=cv2.INTER_LINEAR)
im2_bli_5 = cv2.resize(im_2, dsize=(128, 256), interpolation=cv2.INTER_LINEAR)
im2_bli_6 = cv2.resize(im_2, dsize=(64, 512), interpolation=cv2.INTER_LINEAR)

In [ ]:
# plot
fig = plt.figure(figsize=(16, 8), layout="compressed")
gs = GridSpec(2, 4, figure=fig)
axes = [fig.add_subplot(i) for i in [gs[:, 0], gs[0, 1], gs[0, 2], gs[0, 3], gs[1, 1], gs[1, 2], gs[1, 3]]]
images = [im_2, im2_bli_1, im2_bli_2, im2_bli_3, im2_bli_4, im2_bli_5, im2_bli_6]
titles = [
    f"Original {im_2.shape[:2]}",
    f"{im2_bli_1.shape[:2]} [down scaled]",
    f"{im2_bli_2.shape[:2]} [down scaled]",
    f"{im2_bli_3.shape[:2]} [up Scaled]",
    im2_bli_4.shape[:2],
    im2_bli_5.shape[:2],
    im2_bli_6.shape[:2],
]
for ax, img, title in zip(axes, images, titles):
    ax.imshow(img, cmap="gray", vmin=0, vmax=255)
    ax.set_title(title)
plt.show()

### <a id='toc3_2_3_'></a>[Using PIL](#toc0_)


In [ ]:
im2_bli_1 = img_2.resize(size=(128, 128), resample=Image.Resampling.BILINEAR)
im2_bli_2 = img_2.resize(size=(32, 32), resample=Image.Resampling.BILINEAR)
im2_bli_3 = img_2.resize(size=(555, 555), resample=Image.Resampling.BILINEAR)
im2_bli_4 = img_2.resize(size=(256, 128), resample=Image.Resampling.BILINEAR)
im2_bli_5 = img_2.resize(size=(128, 256), resample=Image.Resampling.BILINEAR)
im2_bli_6 = img_2.resize(size=(64, 512), resample=Image.Resampling.BILINEAR)

In [ ]:
# plot
fig = plt.figure(figsize=(16, 8), layout="compressed")
gs = GridSpec(2, 4, figure=fig)
axes = [fig.add_subplot(i) for i in [gs[:, 0], gs[0, 1], gs[0, 2], gs[0, 3], gs[1, 1], gs[1, 2], gs[1, 3]]]
images = [im_2, im2_bli_1, im2_bli_2, im2_bli_3, im2_bli_4, im2_bli_5, im2_bli_6]
titles = [
    f"Original {im_2.shape[:2]}",
    f"{im2_bli_1.size} [down scaled]",
    f"{im2_bli_2.size} [down scaled]",
    f"{im2_bli_3.size} [up Scaled]",
    im2_bli_4.size,
    im2_bli_5.size,
    im2_bli_6.size,
]
for ax, img, title in zip(axes, images, titles):
    ax.imshow(img, cmap="gray", vmin=0, vmax=255)
    ax.set_title(title)
plt.show()

## <a id='toc3_3_'></a>[Fourier Transform Interpolation](#toc0_)


### <a id='toc3_3_1_'></a>[Manual](#toc0_)


In [ ]:
def fourier_transform_interpolation(image: np.ndarray, new_height: int, new_width: int) -> np.ndarray:
    old_height, old_width = image.shape[:2]

    fft_image_old_shifted = np.fft.fftshift(np.fft.fft2(image))
    fft_image_new_shifted = np.zeros((new_height, new_width), dtype=np.complex128)

    # compute slicing indices for centered placement
    min_h, min_w = min(old_height, new_height), min(old_width, new_width)
    start_h_old, start_h_new = (old_height - min_h) // 2, (new_height - min_h) // 2
    start_w_old, start_w_new = (old_width - min_w) // 2, (new_width - min_w) // 2

    # copy relevant portion of frequency domain
    fft_image_new_shifted[start_h_new : start_h_new + min_h, start_w_new : start_w_new + min_w] = fft_image_old_shifted[
        start_h_old : start_h_old + min_h, start_w_old : start_w_old + min_w
    ]

    # perform inverse FFT and normalize output
    ifft_image_new = np.fft.ifft2(np.fft.ifftshift(fft_image_new_shifted)).real
    new_image = 255 * (ifft_image_new - ifft_image_new.min()) / (ifft_image_new.max() - ifft_image_new.min())

    return new_image.astype(np.uint8)

In [ ]:
im1_fti_1 = fourier_transform_interpolation(im_1, new_height=128, new_width=128)
im1_fti_2 = fourier_transform_interpolation(im_1, new_height=32, new_width=32)
im1_fti_3 = fourier_transform_interpolation(im_1, new_height=555, new_width=555)
im1_fti_4 = fourier_transform_interpolation(im_1, new_height=256, new_width=128)
im1_fti_5 = fourier_transform_interpolation(im_1, new_height=128, new_width=256)
im1_fti_6 = fourier_transform_interpolation(im_1, new_height=64, new_width=512)

In [ ]:
# plot
fig = plt.figure(figsize=(16, 8), layout="compressed")
gs = GridSpec(2, 4, figure=fig)
axes = [fig.add_subplot(i) for i in [gs[:, 0], gs[0, 1], gs[0, 2], gs[0, 3], gs[1, 1], gs[1, 2], gs[1, 3]]]
images = [im_1, im1_fti_1, im1_fti_2, im1_fti_3, im1_fti_4, im1_fti_5, im1_fti_6]
titles = [
    f"Original {im_1.shape}",
    f"{im1_fti_1.shape} [down scaled]",
    f"{im1_fti_2.shape} [down scaled]",
    f"{im1_fti_3.shape} [up Scaled]",
    f"{im1_fti_4.shape}",
    f"{im1_fti_5.shape}",
    f"{im1_fti_6.shape}",
]
for ax, img, title in zip(axes, images, titles):
    ax.imshow(img, cmap="gray", vmin=0, vmax=255)
    ax.set_title(title)
plt.show()

## <a id='toc3_4_'></a>[Comparison](#toc0_)


In [ ]:
# several interpolations [down scale]
im1_nni_downscale = img_1.resize((100, 100), Image.Resampling.NEAREST)
im1_bli_downscale = img_1.resize((100, 100), Image.Resampling.BILINEAR)
im1_bci_downscale = img_1.resize((100, 100), Image.Resampling.BICUBIC)
im1_li_downscale = img_1.resize((100, 100), Image.Resampling.LANCZOS)

# plot
fig, gs = plt.figure(figsize=(16, 8), layout="compressed"), GridSpec(2, 4, figure=plt.gcf())
axes = [fig.add_subplot(i) for i in [gs[:, 0], gs[0, 1], gs[0, 2], gs[1, 1], gs[1, 2]]]
titles = ["Original", "nearest", "bilinear", "bicubic", "lanczos"]
images = [img_1, im1_nni_downscale, im1_bli_downscale, im1_bci_downscale, im1_li_downscale]
for ax, img, title in zip(axes, images, titles):
    ax.imshow(img, cmap="gray", vmin=0, vmax=255)
    ax.set_title(title)
plt.show()

In [ ]:
im1_pil_2 = Image.fromarray(im_1[40:90, 100:150])

# several interpolations [up scale]
im1_nni_upscale = im1_pil_2.resize((256, 256), Image.Resampling.NEAREST)
im1_bli_upscale = im1_pil_2.resize((256, 256), Image.Resampling.BILINEAR)
im1_bci_upscale = im1_pil_2.resize((256, 256), Image.Resampling.BICUBIC)
im1_li_upscale = im1_pil_2.resize((256, 256), Image.Resampling.LANCZOS)

# plot
fig, gs = plt.figure(figsize=(16, 8), layout="compressed"), GridSpec(2, 4, figure=plt.gcf())
axes = [fig.add_subplot(i) for i in [gs[:, 0], gs[0, 1], gs[0, 2], gs[1, 1], gs[1, 2]]]
titles = ["Original", "nearest", "bilinear", "bicubic", "lanczos"]
images = [im1_pil_2, im1_nni_upscale, im1_bli_upscale, im1_bci_upscale, im1_li_upscale]
for ax, img, title in zip(axes, images, titles):
    ax.imshow(img, cmap="gray", vmin=0, vmax=255)
    ax.set_title(title)
plt.show()